In [19]:
import pandas as pd
#1. 데이터 불러오기 및 전처리
#파일 넣기기
input_file = "서울대기오염_2019.xlsx"

#1. 파일 확인: 
df = pd.read_excel(input_file)

print(df.info())
print(df.isnull().sum())

df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9491 entries, 0 to 9490
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   날짜               9491 non-null   object 
 1   측정소명             9491 non-null   object 
 2   미세먼지             9278 non-null   float64
 3   초미세먼지            9288 non-null   float64
 4   오존               9294 non-null   float64
 5   이산화질소
NO2 (ppm)  9285 non-null   float64
 6   일산화탄소
CO (ppm)   9234 non-null   float64
 7   아황산가스
SO2(ppm)   9298 non-null   float64
dtypes: float64(6), object(2)
memory usage: 593.3+ KB
None
날짜                    0
측정소명                  0
미세먼지                213
초미세먼지               203
오존                  197
이산화질소\nNO2 (ppm)    206
일산화탄소\nCO (ppm)     257
아황산가스\nSO2(ppm)     193
dtype: int64


,날짜,측정소명,미세먼지,초미세먼지,오존,이산화질소\nNO2 (ppm),일산화탄소\nCO (ppm),아황산가스\nSO2(ppm)
0,전체,평균,42.0,25.0,0.025,0.028,0.5,0.004
1,2019-12-31,평균,26.0,15.0,0.022,0.016,0.4,0.003
2,2019-12-31,강남구,22.0,14.0,0.025,0.014,0.4,0.003
3,2019-12-31,강동구,27.0,19.0,0.019,0.020,0.4,0.003
4,2019-12-31,강북구,31.0,17.0,0.022,0.022,0.4,0.002
...,...,...,...,...,...,...,...,...
9486,2019-01-01,용산구,30.0,24.0,0.014,0.032,0.6,0.003
9487,2019-01-01,은평구,33.0,22.0,0.020,0.023,0.9,0.005
9488,2019-01-01,종로구,34.0,21.0,0.017,0.031,0.7,0.004
9489,2019-01-01,중구,36.0,25.0,0.020,0.030,0.6,0.003


In [20]:
# [1-1] 데이터프레임으로 불러오기
# [1-2] 분석변수만 추출 및 컬럼명 변경: date, district, pm10, pm25

# [1-2] 분석변수 추출 및 컬럼명 변경
df_filtered = df[['날짜', '측정소명', '미세먼지', '초미세먼지']].copy()
df_filtered.columns = ['date', 'district', 'pm10', 'pm25']

df_filtered.isnull().sum()



date          0
district      0
pm10        213
pm25        203
dtype: int64

In [ ]:
# [1-3] 결측치 확인 및 제거 (df_filtered 기준)
# df_filtered는 ['date', 'district', 'pm10', 'pm25'] 컬럼만 존재함

# 결측치가 있는 열 목록만 추출 (pm10, pm25만 해당)
columns_with_na = df_filtered.columns[df_filtered.isnull().any()].tolist()

# 'district'별 평균값으로 결측치 채우기
for col in columns_with_na:
    df_filtered[col] = df_filtered.groupby('district')[col].transform(lambda x: x.fillna(x.mean()))

# 결측치 확인
print(df_filtered.isnull().sum())



date        0
district    0
pm10        0
pm25        0
dtype: int64


In [24]:
# [1-4] 자료형 변환

# 1. 'date' 컬럼을 datetime 형식으로 변환
df_filtered['date'] = pd.to_datetime(df_filtered['date'], errors='coerce')

# 2. pm10, pm25 컬럼을 실수형으로 변환 (혹시 모를 문자 포함 대비)
df_filtered['pm10'] = pd.to_numeric(df_filtered['pm10'], errors='coerce')
df_filtered['pm25'] = pd.to_numeric(df_filtered['pm25'], errors='coerce')


df_filtered


,date,district,pm10,pm25
0,NaT,평균,42.0,25.0
1,2019-12-31,평균,26.0,15.0
2,2019-12-31,강남구,22.0,14.0
3,2019-12-31,강동구,27.0,19.0
4,2019-12-31,강북구,31.0,17.0
...,...,...,...,...
9486,2019-01-01,용산구,30.0,24.0
9487,2019-01-01,은평구,33.0,22.0
9488,2019-01-01,종로구,34.0,21.0
9489,2019-01-01,중구,36.0,25.0


In [25]:
#2. 파생변수수

# [2-1] month, day 파생변수 생성
df_filtered['month'] = df_filtered['date'].dt.month

# [2-1] 요일 (숫자형: 월=0, 일=6)
df_filtered['day'] = df_filtered['date'].dt.dayofweek


df_filtered

,date,district,pm10,pm25,month,day
0,NaT,평균,42.0,25.0,NaN,NaN
1,2019-12-31,평균,26.0,15.0,12.0,1.0
2,2019-12-31,강남구,22.0,14.0,12.0,1.0
3,2019-12-31,강동구,27.0,19.0,12.0,1.0
4,2019-12-31,강북구,31.0,17.0,12.0,1.0
...,...,...,...,...,...,...
9486,2019-01-01,용산구,30.0,24.0,1.0,1.0
9487,2019-01-01,은평구,33.0,22.0,1.0,1.0
9488,2019-01-01,종로구,34.0,21.0,1.0,1.0
9489,2019-01-01,중구,36.0,25.0,1.0,1.0


In [27]:
# [2-2] 계절(season) 변수 생성

# 월(month) 컬럼 생성
df_filtered['month'] = df_filtered['date'].dt.month

# 계절(season) 컬럼 생성
def get_season(month):
    if month in [3, 4, 5]:
        return 'spring'
    elif month in [6, 7, 8]:
        return 'summer'
    elif month in [9, 10, 11]:
        return 'autumn'
    else:
        return 'winter'

df_filtered['season'] = df_filtered['month'].apply(get_season)

df_filtered


,date,district,pm10,pm25,month,day,season
0,NaT,평균,42.0,25.0,NaN,NaN,winter
1,2019-12-31,평균,26.0,15.0,12.0,1.0,winter
2,2019-12-31,강남구,22.0,14.0,12.0,1.0,winter
3,2019-12-31,강동구,27.0,19.0,12.0,1.0,winter
4,2019-12-31,강북구,31.0,17.0,12.0,1.0,winter
...,...,...,...,...,...,...,...
9486,2019-01-01,용산구,30.0,24.0,1.0,1.0,winter
9487,2019-01-01,은평구,33.0,22.0,1.0,1.0,winter
9488,2019-01-01,종로구,34.0,21.0,1.0,1.0,winter
9489,2019-01-01,중구,36.0,25.0,1.0,1.0,winter


In [28]:
#3. 전처리 완료 데이터 확인 및 저장

# [3-1] 최종 분석 대상 데이터 확인
df_filtered
# [3-2] 'card_output.csv'로 저장
df.to_csv('card_output.csv')


In [29]:
# 데이터 분석 단계 4. 연간 미세먼지 평균 구하기

# [4-1] 전체 데이터 기준 PM10 평균

pm10_mean = df_filtered['pm10'].mean()
# 분석결과 작성
print(f"PM10의 전체 평균값: {pm10_mean:.2f} ㎍/m³")


PM10의 전체 평균값: 41.78 ㎍/m³


In [30]:
#5. 미세먼지 최댓값 날짜 확인
# [5-1] PM10 최댓값이 발생한 날짜, 구 출력
maximum_pm10 = df_filtered.loc[df_filtered['pm10'].idxmax()]

# 출력
print(f"PM10 최댓값: {maximum_pm10['pm10']} ㎍/m³")
print(f"날짜: {maximum_pm10['date'].date()}")
print(f"측정소(구): {maximum_pm10['district']}")



PM10 최댓값: 228.0 ㎍/m³
날짜: 2019-03-05
측정소(구): 강북구


In [32]:
# [6-1] 각 구별 PM10 평균 계산
pm10_district = df_filtered.groupby('district')['pm10'].mean().reset_index()

# 컬럼명 변경
pm10_district.columns = ['district', 'mean_pm10']

# [6-2] 상위 5개 구 출력
top5= pm10_district.sort_values(by='mean_pm10', ascending=False).head(5)

print(top5)

   district  mean_pm10
4       관악구  48.974359
18      양천구  47.695055
12      마포구  47.086154
3       강서구  46.504132
2       강북구  44.950685


In [34]:
# [7-1] 계절별 평균 pm10, pm25 동시 계산
seasonal_mean = df_filtered.groupby('season')[['pm10', 'pm25']].mean().reset_index()

# 컬럼명 변경
seasonal_mean.columns = ['season', 'mean_pm10', 'mean_pm25']

# [7-2] PM10 평균값 기준 오름차순 정렬
seasonal_mean_sorted = seasonal_mean.sort_values(by='mean_pm10', ascending=True)

# 출력
print(seasonal_mean_sorted)


   season  mean_pm10  mean_pm25
2  summer  26.519951  18.189842
0  autumn  32.076626  16.505728
1  spring  54.088629  31.558110
3  winter  54.618362  33.620668
